In [1]:
%matplotlib inline

/home/jrudra/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
from __future__ import division,print_function

import os, json
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

In [3]:
import utils; reload(utils)
from utils import plots

Using TensorFlow backend.


In [4]:
from keras import backend as K
from keras.models import Model
from keras.layers.core import Dense, Lambda
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.applications.xception import Xception
from keras.callbacks import ModelCheckpoint

# In case we are going to use the TensorFlow backend we need to explicitly set the tensorflow image ordering
from keras import backend as K
K.set_image_dim_ordering('tf')

In [5]:
%%javascript
// Source - https://stackoverflow.com/a/23395136

Jupyter.beep = () => {
    var snd = new Audio("data:audio/wav;base64,//uQRAAAAWMSLwUIYAAsYkXgoQwAEaYLWfkWgAI0wWs/ItAAAGDgYtAgAyN+QWaAAihwMWm4G8QQRDiMcCBcH3Cc+CDv/7xA4Tvh9Rz/y8QADBwMWgQAZG/ILNAARQ4GLTcDeIIIhxGOBAuD7hOfBB3/94gcJ3w+o5/5eIAIAAAVwWgQAVQ2ORaIQwEMAJiDg95G4nQL7mQVWI6GwRcfsZAcsKkJvxgxEjzFUgfHoSQ9Qq7KNwqHwuB13MA4a1q/DmBrHgPcmjiGoh//EwC5nGPEmS4RcfkVKOhJf+WOgoxJclFz3kgn//dBA+ya1GhurNn8zb//9NNutNuhz31f////9vt///z+IdAEAAAK4LQIAKobHItEIYCGAExBwe8jcToF9zIKrEdDYIuP2MgOWFSE34wYiR5iqQPj0JIeoVdlG4VD4XA67mAcNa1fhzA1jwHuTRxDUQ//iYBczjHiTJcIuPyKlHQkv/LHQUYkuSi57yQT//uggfZNajQ3Vmz+Zt//+mm3Wm3Q576v////+32///5/EOgAAADVghQAAAAA//uQZAUAB1WI0PZugAAAAAoQwAAAEk3nRd2qAAAAACiDgAAAAAAABCqEEQRLCgwpBGMlJkIz8jKhGvj4k6jzRnqasNKIeoh5gI7BJaC1A1AoNBjJgbyApVS4IDlZgDU5WUAxEKDNmmALHzZp0Fkz1FMTmGFl1FMEyodIavcCAUHDWrKAIA4aa2oCgILEBupZgHvAhEBcZ6joQBxS76AgccrFlczBvKLC0QI2cBoCFvfTDAo7eoOQInqDPBtvrDEZBNYN5xwNwxQRfw8ZQ5wQVLvO8OYU+mHvFLlDh05Mdg7BT6YrRPpCBznMB2r//xKJjyyOh+cImr2/4doscwD6neZjuZR4AgAABYAAAABy1xcdQtxYBYYZdifkUDgzzXaXn98Z0oi9ILU5mBjFANmRwlVJ3/6jYDAmxaiDG3/6xjQQCCKkRb/6kg/wW+kSJ5//rLobkLSiKmqP/0ikJuDaSaSf/6JiLYLEYnW/+kXg1WRVJL/9EmQ1YZIsv/6Qzwy5qk7/+tEU0nkls3/zIUMPKNX/6yZLf+kFgAfgGyLFAUwY//uQZAUABcd5UiNPVXAAAApAAAAAE0VZQKw9ISAAACgAAAAAVQIygIElVrFkBS+Jhi+EAuu+lKAkYUEIsmEAEoMeDmCETMvfSHTGkF5RWH7kz/ESHWPAq/kcCRhqBtMdokPdM7vil7RG98A2sc7zO6ZvTdM7pmOUAZTnJW+NXxqmd41dqJ6mLTXxrPpnV8avaIf5SvL7pndPvPpndJR9Kuu8fePvuiuhorgWjp7Mf/PRjxcFCPDkW31srioCExivv9lcwKEaHsf/7ow2Fl1T/9RkXgEhYElAoCLFtMArxwivDJJ+bR1HTKJdlEoTELCIqgEwVGSQ+hIm0NbK8WXcTEI0UPoa2NbG4y2K00JEWbZavJXkYaqo9CRHS55FcZTjKEk3NKoCYUnSQ0rWxrZbFKbKIhOKPZe1cJKzZSaQrIyULHDZmV5K4xySsDRKWOruanGtjLJXFEmwaIbDLX0hIPBUQPVFVkQkDoUNfSoDgQGKPekoxeGzA4DUvnn4bxzcZrtJyipKfPNy5w+9lnXwgqsiyHNeSVpemw4bWb9psYeq//uQZBoABQt4yMVxYAIAAAkQoAAAHvYpL5m6AAgAACXDAAAAD59jblTirQe9upFsmZbpMudy7Lz1X1DYsxOOSWpfPqNX2WqktK0DMvuGwlbNj44TleLPQ+Gsfb+GOWOKJoIrWb3cIMeeON6lz2umTqMXV8Mj30yWPpjoSa9ujK8SyeJP5y5mOW1D6hvLepeveEAEDo0mgCRClOEgANv3B9a6fikgUSu/DmAMATrGx7nng5p5iimPNZsfQLYB2sDLIkzRKZOHGAaUyDcpFBSLG9MCQALgAIgQs2YunOszLSAyQYPVC2YdGGeHD2dTdJk1pAHGAWDjnkcLKFymS3RQZTInzySoBwMG0QueC3gMsCEYxUqlrcxK6k1LQQcsmyYeQPdC2YfuGPASCBkcVMQQqpVJshui1tkXQJQV0OXGAZMXSOEEBRirXbVRQW7ugq7IM7rPWSZyDlM3IuNEkxzCOJ0ny2ThNkyRai1b6ev//3dzNGzNb//4uAvHT5sURcZCFcuKLhOFs8mLAAEAt4UWAAIABAAAAAB4qbHo0tIjVkUU//uQZAwABfSFz3ZqQAAAAAngwAAAE1HjMp2qAAAAACZDgAAAD5UkTE1UgZEUExqYynN1qZvqIOREEFmBcJQkwdxiFtw0qEOkGYfRDifBui9MQg4QAHAqWtAWHoCxu1Yf4VfWLPIM2mHDFsbQEVGwyqQoQcwnfHeIkNt9YnkiaS1oizycqJrx4KOQjahZxWbcZgztj2c49nKmkId44S71j0c8eV9yDK6uPRzx5X18eDvjvQ6yKo9ZSS6l//8elePK/Lf//IInrOF/FvDoADYAGBMGb7FtErm5MXMlmPAJQVgWta7Zx2go+8xJ0UiCb8LHHdftWyLJE0QIAIsI+UbXu67dZMjmgDGCGl1H+vpF4NSDckSIkk7Vd+sxEhBQMRU8j/12UIRhzSaUdQ+rQU5kGeFxm+hb1oh6pWWmv3uvmReDl0UnvtapVaIzo1jZbf/pD6ElLqSX+rUmOQNpJFa/r+sa4e/pBlAABoAAAAA3CUgShLdGIxsY7AUABPRrgCABdDuQ5GC7DqPQCgbbJUAoRSUj+NIEig0YfyWUho1VBBBA//uQZB4ABZx5zfMakeAAAAmwAAAAF5F3P0w9GtAAACfAAAAAwLhMDmAYWMgVEG1U0FIGCBgXBXAtfMH10000EEEEEECUBYln03TTTdNBDZopopYvrTTdNa325mImNg3TTPV9q3pmY0xoO6bv3r00y+IDGid/9aaaZTGMuj9mpu9Mpio1dXrr5HERTZSmqU36A3CumzN/9Robv/Xx4v9ijkSRSNLQhAWumap82WRSBUqXStV/YcS+XVLnSS+WLDroqArFkMEsAS+eWmrUzrO0oEmE40RlMZ5+ODIkAyKAGUwZ3mVKmcamcJnMW26MRPgUw6j+LkhyHGVGYjSUUKNpuJUQoOIAyDvEyG8S5yfK6dhZc0Tx1KI/gviKL6qvvFs1+bWtaz58uUNnryq6kt5RzOCkPWlVqVX2a/EEBUdU1KrXLf40GoiiFXK///qpoiDXrOgqDR38JB0bw7SoL+ZB9o1RCkQjQ2CBYZKd/+VJxZRRZlqSkKiws0WFxUyCwsKiMy7hUVFhIaCrNQsKkTIsLivwKKigsj8XYlwt/WKi2N4d//uQRCSAAjURNIHpMZBGYiaQPSYyAAABLAAAAAAAACWAAAAApUF/Mg+0aohSIRobBAsMlO//Kk4soosy1JSFRYWaLC4qZBYWFRGZdwqKiwkNBVmoWFSJkWFxX4FFRQWR+LsS4W/rFRb/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////VEFHAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAU291bmRib3kuZGUAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAMjAwNGh0dHA6Ly93d3cuc291bmRib3kuZGUAAAAAAAAAACU=");  
    snd.play();
}

Jupyter.beep();

<IPython.core.display.Javascript object>

In [6]:
# As large as you can, but no larger than 64 is recommended. 
# If you have an older or cheaper GPU, you'll run out of memory, so will have to decrease this.
batch_size=24
path = "/home/jrudra/staging/jrudra/kaggle/plant_seedlings/data/"

In [7]:
def vgg_preprocess(x):
    """
        Subtracts the mean RGB value, and transposes RGB to BGR.
        The mean RGB was computed on the image set used to train the VGG model.

        Args: 
            x: Image array (height x width x channels)
        Returns:
            Image array (height x width x transposed_channels)
    """
    # vgg_mean shape varies depending on image_data_format besing used (tf or th)
    vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((1,1,3))
    x = x - vgg_mean
    return x   #[:, ::-1] # reverse axis rgb->bgr

In [8]:
def get_batches(path, gen=image.ImageDataGenerator(), shuffle=True, batch_size=8, class_mode='categorical'):
    """ 
        Takes the path to a directory, and generates batches of augmented/normalized data. 
        Yields batches indefinitely, in an infinite loop.

        See Keras documentation: https://keras.io/preprocessing/image/
    """ 
    return gen.flow_from_directory(path, target_size=(224,224),
            class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [9]:
# Grab a few images at a time for training and validation.
# NB: They must be in subdirectories named based on their category
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2)

Found 4269 images belonging to 12 classes.
Found 481 images belonging to 12 classes.


In [10]:
Xception?

In [11]:
base_model = Xception(include_top=False, weights='imagenet', input_shape=(224,224,3))

In [12]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [13]:
x = base_model.output

In [14]:
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(batches.num_classes, activation='softmax')(x)

In [15]:
model = Model(inputs=base_model.input, outputs=predictions)

In [16]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [17]:
for layer in base_model.layers:
    layer.trainable = False

In [18]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
classes = list(iter(batches.class_indices)) # get a list of all the class labels
        
# batches.class_indices is a dict with the class name as key and an index as value
# eg. {'cats': 0, 'dogs': 1}

# sort the class labels by index according to batches.class_indices and update model.classes
for c in batches.class_indices:
    classes[batches.class_indices[c]] = c

classes

['Black-grass',
 'Charlock',
 'Cleavers',
 'Common Chickweed',
 'Common wheat',
 'Fat Hen',
 'Loose Silky-bent',
 'Maize',
 'Scentless Mayweed',
 'Shepherds Purse',
 'Small-flowered Cranesbill',
 'Sugar beet']

In [20]:
checkpoint = ModelCheckpoint(filepath='./models/xception/weights.{epoch:02d}-{val_acc:.4f}.hdf5', monitor='val_acc',
                            save_weights_only=True, mode='max', verbose=1)

In [21]:
# Fits the model on data yielded batch-by-batch by a Python generator.
model.fit_generator(generator=batches, steps_per_epoch=len(batches), epochs=10, callbacks=[checkpoint],
                validation_data=val_batches, validation_steps=len(val_batches), shuffle=True, initial_epoch=0)

Epoch 1/10
178/178 [==============================] - 59s 333ms/step - loss: 1.2280 - acc: 0.6095 - val_loss: 1.3494 - val_acc: 0.5884
Epoch 2/10
178/178 [==============================] - 52s 290ms/step - loss: 0.7223 - acc: 0.7653 - val_loss: 0.8014 - val_acc: 0.7277
Epoch 3/10
178/178 [==============================] - 52s 290ms/step - loss: 0.5731 - acc: 0.8122 - val_loss: 0.7538 - val_acc: 0.7505
Epoch 4/10
178/178 [==============================] - 56s 316ms/step - loss: 0.4964 - acc: 0.8314 - val_loss: 0.6795 - val_acc: 0.7963
Epoch 5/10
178/178 [==============================] - 52s 294ms/step - loss: 0.4429 - acc: 0.8480 - val_loss: 0.6715 - val_acc: 0.7775
Epoch 6/10
178/178 [==============================] - 58s 326ms/step - loss: 0.3745 - acc: 0.8765 - val_loss: 0.6869 - val_acc: 0.7879
Epoch 7/10
178/178 [==============================] - 51s 284ms/step - loss: 0.3687 - acc: 0.8719 - val_loss: 0.7108 - val_acc: 0.8087
Epoch 8/10
178/178 [==============================] - 5

In [22]:
test_batches = get_batches(path + os.sep + 'test', shuffle=False)

Found 794 images belonging to 1 classes.


In [23]:
def pred_batch(model, imgs, filenames):
    preds = model.predict(imgs)
    idxs = np.argmax(preds, axis=1)

    #print('Shape: {}'.format(preds.shape))
    #print('First 5 classes: {}'.format(classes[:5]))
    #print('First 5 probabilities: {}\n'.format(preds[0, :5]))
    #print('filename, Predictions prob, class: ')
    temp = list()
    
    for i in range(len(idxs)):
        idx = idxs[i]
        #print ('{}, {:.4f}, {}'.format(test_batches.filenames[i], preds[i, idx], classes[idx]))
        print([filenames[i], str(preds[i, idx]), classes[idx]])
        temp.append([filenames[i], str(preds[i, idx]), classes[idx]])
    
    return(temp)

In [24]:
prediction_list = list()

In [25]:
for batch in range(len(test_batches)):
    
    imgs, labels = next(test_batches)
    filenames = test_batches.filenames[(8*batch): (8 + 8*batch)]
    temp_list = pred_batch(model, imgs, filenames)
    prediction_list.append(temp_list)

['Maize/0021e90e4.png', '0.999998', 'Small-flowered Cranesbill']
['Maize/003d61042.png', '0.979056', 'Fat Hen']
['Maize/007b3da8b.png', '0.99665', 'Sugar beet']
['Maize/0086a6340.png', '0.999568', 'Common Chickweed']
['Maize/00c47e980.png', '0.999593', 'Sugar beet']
['Maize/00d090cde.png', '0.752221', 'Loose Silky-bent']
['Maize/00ef713a8.png', '0.868641', 'Fat Hen']
['Maize/01291174f.png', '0.965518', 'Fat Hen']
['Maize/026716f9b.png', '0.966296', 'Loose Silky-bent']
['Maize/02cfeb38d.png', '0.991115', 'Loose Silky-bent']
['Maize/03566743d.png', '0.999928', 'Fat Hen']
['Maize/03a2ee656.png', '0.99982', 'Small-flowered Cranesbill']
['Maize/03e322a29.png', '0.723729', 'Fat Hen']
['Maize/03ef36742.png', '0.992532', 'Scentless Mayweed']
['Maize/043449b0b.png', '0.999956', 'Sugar beet']
['Maize/0437393b1.png', '0.715241', 'Sugar beet']
['Maize/04814f36d.png', '0.504026', 'Loose Silky-bent']
['Maize/05341a8a6.png', '0.740932', 'Scentless Mayweed']
['Maize/060450d79.png', '0.455352', 'Common

In [26]:
flat_list = [item for sublist in prediction_list for item in sublist]

In [27]:
import pandas as pd
df = pd.DataFrame(flat_list, columns=['file', 'Predictions prob', 'species'])
df.head()

,file,Predictions prob,species
0,Maize/0021e90e4.png,0.999998,Small-flowered Cranesbill
1,Maize/003d61042.png,0.979056,Fat Hen
2,Maize/007b3da8b.png,0.99665,Sugar beet
3,Maize/0086a6340.png,0.999568,Common Chickweed
4,Maize/00c47e980.png,0.999593,Sugar beet


In [28]:
df = df[['file','species']]
df.head()

,file,species
0,Maize/0021e90e4.png,Small-flowered Cranesbill
1,Maize/003d61042.png,Fat Hen
2,Maize/007b3da8b.png,Sugar beet
3,Maize/0086a6340.png,Common Chickweed
4,Maize/00c47e980.png,Sugar beet


In [32]:
for i in range(df.shape[0]):
    df.file[i] = os.path.basename(df.file[i])
df.head()

,file,species
0,0021e90e4.png,Small-flowered Cranesbill
1,003d61042.png,Fat Hen
2,007b3da8b.png,Sugar beet
3,0086a6340.png,Common Chickweed
4,00c47e980.png,Sugar beet


In [30]:
df.head()

,file,species
0,Maize/0021e90e4.png,Small-flowered Cranesbill
1,Maize/003d61042.png,Fat Hen
2,Maize/007b3da8b.png,Sugar beet
3,Maize/0086a6340.png,Common Chickweed
4,Maize/00c47e980.png,Sugar beet


In [33]:
df.shape

(794, 2)

In [34]:
from IPython.lib.display import FileLink

In [35]:
df.to_csv('./../data/submissions/xcpetion_epoch10_results.csv', index=False)

In [36]:
FileLink('./../data/submissions/xcpetion_epoch10_results.csv')

/media/philips/e2cc7e65-4ca5-4332-9fbb-ba879e676ae3/staging/jrudra/kaggle/plant_seedlings/data/submissions/xcpetion_epoch10_results.csv